# 一、训练流程

CRF++，是Taku Kudo于2005年用C++实现并开源

我们拿分词来作为例子

1. 准备样本数据
2. 生成特征函数
3. 计算特征得分
4. 计算梯度（每一次迭代全部样本，进行计算梯度）
5. 不断迭代训练

# 二、开启多线程

目前CRF++支持两种训练算法，一种是拟牛顿算法中的LBFGS算法(批处理优化的ML问题中常用, 缺点是收敛速度慢)，另一种是MIRA算法， 本篇文章主要探讨LBFGS算法的实现过程。在learn函数中，训练算法的入口代码如下：

```c++
switch (algorithm) {
    case MIRA:　　　　　　　　　　　　　　　　　　　　//MIRA算法的入口
      if (!runMIRA(x, &feature_index, &alpha[0],
                   maxitr, C, eta, shrinking_size, thread_num)) {
        WHAT_ERROR("MIRA execute error");
      }
      break;
    case CRF_L2:　　　　　　　　　　　　　　　　　　//LBFGS-L2正则化的入口函数
      if (!runCRF(x, &feature_index, &alpha[0],
                  maxitr, C, eta, shrinking_size, thread_num, false)) {
        WHAT_ERROR("CRF_L2 execute error");
      }
      break;
    case CRF_L1:　　　　　　　　　　　　　　　　　 //LBFGS-L1正则化的入口函数
      if (!runCRF(x, &feature_index, &alpha[0],
                  maxitr, C, eta, shrinking_size, thread_num, true)) {
        WHAT_ERROR("CRF_L1 execute error");
      }
      break;
  }
```

runCRF函数中会初始化CRFEncoderThread数组，并启动每个线程，源码如下：

```c++
bool runCRF(const std::vector<TaggerImpl* > &x,
            EncoderFeatureIndex *feature_index,
            double *alpha,
            size_t maxitr,
            float C,
            double eta,
            unsigned short shrinking_size,
            unsigned short thread_num,
            bool orthant) {
  ... //省略代码
　for (size_t itr = 0; itr < maxitr; ++itr) { //开始迭代， 最大迭代次数为maxitr，即命令行参数-m，默认10k
    for (size_t i = 0; i < thread_num; ++i) {
      thread[i].start();                    //启动每个线程，start函数中会调用CRFEncoderThread类中的run函数  
    }
 
    for (size_t i = 0; i < thread_num; ++i) {
      thread[i].join();　　　　　　　　　　　　//等待所有线程结束
    }
　　... //省略代码
```

CRFEncoderThread类中的run函数调用gradient函数，来计算梯度，完成一系列的核心计算。源码如下：

```c++
void run() {
    obj = 0.0;
    err = zeroone = 0;
    std::fill(expected.begin(), expected.end(), 0.0); //excepted变量存放期望
    for (size_t i = start_i; i < size; i += thread_num) {//每个线程并行处理多个句子， 并且每个线程处理的句子不相同， size是句子的个数
      obj += x[i]->gradient(&expected[0]);  //x[i]是TaggerImpl对象，代表一个句子， gradient函数主要功能： 1. 构建无向图  2. 调用前向后向算法 3. 计算期望
      int error_num = x[i]->eval();         
      err += error_num;
      if (error_num) {
        ++zeroone;
      }
    }
  }
```

# 三、训练数据和特征函数


## 1、训练数据格式

```txt
那	S
音	B
韵	E
如	S
轻	B
柔	E
的	S
夜	B
风	E
，	S
 
惊	S
溅	S
起	S
不	B
可	M
言	M
传	E
的	S
天	B
籁	E
。	S
```
使用\t隔开即可。

## 2、crf++使用特征模板产生特征函数

```txt
# Unigram
U00:%x[-2,0]
U01:%x[-1,0]
U02:%x[0,0]
U03:%x[1,0]
U04:%x[2,0]
U05:%x[-2,0]/%x[-1,0]/%x[0,0]
U06:%x[-1,0]/%x[0,0]/%x[1,0]
U07:%x[0,0]/%x[1,0]/%x[2,0]
U08:%x[-1,0]/%x[0,0]
U09:%x[0,0]/%x[1,0]

# Bigram
B
```
**状态特征（对应hmm的发射矩阵，但是利用的上下文特征更多）**

Unigram是二元马尔科夫

U00:%x[-2,0]解释：

代表当前行的第前2行的第1列数据，利用上面例子的样本数据，假设现在该提取“如”字的特征了，那么U00:%x[-2,0]就是“音”字。

那么对应的特征函数就是：

f(s, o)，其中s为t时刻的的标签(output)，o为t时刻的上下文

f('B', '音') = if (output = B and feature="U00:音") return 1 else return 0

**label转移特征（对应hmm的状态转移矩阵）**

Bigram是一元马尔科夫

每一行%x[#,#]生成一个CRFs中的边(Edge)特征函数:f(s', s, o)，其中s'为t – 1时刻的标签。也就是说，Bigram类型与Unigram大致机同，只是还要考虑到t–1时刻的标签。

如果只写一个B的话,默认生成f(s', s)，这意味着前一个output token和current token将组合成bigram features。

f('E', 'S')=if (output = E and next_output = S) return 1 else return 0

**对于角标越界的如何处理？**

1. 可以使用特殊特征【这个是结构化平均感知器里的处理方式】

```java
private static final char CHAR_BEGIN = '\u0001';
private static final char CHAR_END = '\u0002';
```

# 四、构造无向图

我们知道条件随机场是概率图模型，几乎所有的概率计算都是在无向图上进行的。那么这个图是如果构造的呢？答案就在gradient函数第一个调用 —— buildLattice函数中。该函数完成2个核心功能：

1. 构建无向图 

2. 计算节点以及边上的得分

先看一下无向图的构造过程：

```c++
void TaggerImpl::buildLattice() {
  if (x_.empty()) {
    return;
  }
 
  feature_index_->rebuildFeatures(this); //调用该方法初始化节点（Node）和边（Path），并连接
 
  ... //省略代码  
}

void FeatureIndex::rebuildFeatures(TaggerImpl *tagger) const {
  size_t fid = tagger->feature_id();           //取出当前句子的feature_id，上篇介绍构造特征的时候，在buildFeatures函数中会set feature_id
  const size_t thread_id = tagger->thread_id();
 
  Allocator *allocator = tagger->allocator();
  allocator->clear_freelist(thread_id);
  FeatureCache *feature_cache = allocator->feature_cache();
　
  //每个词以及对应的所有可能的label，构造节点
  for (size_t cur = 0; cur < tagger->size(); ++cur) { //遍历每个词， 
    const int *f = (*feature_cache)[fid++]; //取出每个词的特征列表，词的特征列表对应特征模板里的Unigram特征
    for (size_t i = 0; i < y_.size(); ++i) { //每个词都对应不同的label， 每个label用数组的下标表示，每个特征+当前的label就是特征函数
      Node *n = allocator->newNode(thread_id); //初始化新的节点，即Node对象
      n->clear();
      n->x = cur; //当前词
      n->y = i;　　//当前词的label
      n->fvector = f; //特征列表
      tagger->set_node(n, cur, i); //有一个二维数组node_存放每个节点
    }
  }
 
  //从第二个词开始构造节点之间的边，两个词之间有y_.size()*y_.size()条边
  for (size_t cur = 1; cur < tagger->size(); ++cur) {
    const int *f = (*feature_cache)[fid++]; //取出每个边的特征列表，边的特征列表对应特征模板里的Bigram特征
    for (size_t j = 0; j < y_.size(); ++j) {//前一个词的label有y_.size()种情况，即y_.size()个节点
      for (size_t i = 0; i < y_.size(); ++i) {//当前词label也有y_.size()种情况，即y_.size()个节点
        Path *p = allocator->newPath(thread_id);//初始化新的节点，即Path对象
        p->clear();
　　　　 //add函数会设置当前边的左右节点，同时会把当前边加入到左右节点的边集合中
        p->add(tagger->node(cur - 1, j), //前一个节点
               tagger->node(cur,     i)); //当前节点 
        p->fvector = f;
      }
    }
  }
}
```

图构造完成后， 接下来看看节点和边上的得分是如何计算的。那么得分是什么？

就是特征函数值乘以特征的权重。这部分源码在buildLattice函数中，具体如下：

```c++
for (size_t i = 0; i < x_.size(); ++i) {
    for (size_t j = 0; j < ysize_; ++j) {
      feature_index_->calcCost(node_[i][j]); //计算节点的得分
      const std::vector<Path *> &lpath = node_[i][j]->lpath;
      for (const_Path_iterator it = lpath.begin(); it != lpath.end(); ++it) {
        feature_index_->calcCost(*it); //计算边的得分
      }
    }
}
//节点的得分计算函数
void FeatureIndex::calcCost(Node *n) const {
  n->cost = 0.0;
 
#define ADD_COST(T, A)                                                  \
  do { T c = 0;                                                               \
    for (const int *f = n->fvector; *f != -1; ++f) { c += (A)[*f + n->y];  }  \  //取每个特征以及当前节点的label，即为特征函数，且值为1，特征函数乘以权重（alpha_[*f + n->y]）是得分，特征函数为1所以得分=alpha_[*f + n->y]*1，对所有得分求和
    n->cost =cost_factor_ *(T)c; } while (0) //cost_factor_是得分因子
 
  if (alpha_float_) {
    ADD_COST(float,  alpha_float_);
  } else {
    ADD_COST(double, alpha_); //将会在这里调用， 上一篇内容可以看到，CRF++初始化的是alpha_变量
  }
#undef ADD_COST
}
//边的得分计算函数与节点类似，不再赘述
```

# 五、梯度下降，进行权重更新

crf权重更新是针对所有样本数据的一次迭代，如果样本量为m，那么

$$\frac{\partial L}{\partial W_{k}}=\sum_{t=1}^{T}\sum_{k=1}^{K}F_k(y_{t-1}^{(i)},y_{t}^{(i)},x_{t}^{(i)})-\sum_{y}\bigg(P(Y|X)\sum_{t=1}^{T}\sum_{k=1}^{K}F_k(y_{t-1},y_{t},x_{t}^{(i)})\bigg)$$

变为

$$\frac{\partial L}{\partial W_{k}}=\sum_1^m\left[\sum_{t=1}^{T}\sum_{k=1}^{K}F_k(y_{t-1}^{(i)},y_{t}^{(i)},x_{t}^{(i)})-\sum_{y}\bigg(P(Y|X)\sum_{t=1}^{T}\sum_{k=1}^{K}F_k(y_{t-1},y_{t},x_{t}^{(i)})\bigg)\right]$$


利用前向后向算法，进行特征函数的期望计算：

$$\sum_{y}\bigg(P(Y|X)\sum_{t=1}^{T}\sum_{k=1}^{K}F_k(y_{t-1},y_{t},x_{t}^{(i)})\bigg)$$

<font color="red">注意：P(Y|X)指的是一个训练样本，也就是一个句子的类标序列的概率</font>

$$E_{P(Y|X)}(f_k)=\sum_{y}P(y|x){f_k}(y,x)={\sum_{i=1}^{n+1}} \sum_{y_{i-1}\ \ ,y_i} f_k(y_{i-1},y_i,x,i)\bullet{\frac{{\alpha_i}^T(y_{i-1}|x)M_i (y_{i-1},{y_i}|x){\beta_i}(y_i|x)}{Z(x)}}$$

```c++
for (size_t i = 0;   i < x_.size(); ++i) { //遍历每一个节点的，遍历计算每个节点和每条边上的特征函数，计算每个特征函数的期望
    for (size_t j = 0; j < ysize_; ++j) {
      node_[i][j]->calcExpectation(expected, Z_, ysize_);
    }
}
void Node::calcExpectation(double *expected, double Z, size_t size) const { //状态特征的期望
  const double c = std::exp(alpha + beta - cost - Z); //这里减去一个多余的cost，剩下的就是上面提到的节点的概率值 P(Yi=yi | x)，这里已经取了exp，跟矩阵形式的计算结果一致
  for (const int *f = fvector; *f != -1; ++f) { 
    expected[*f + y] += c;           //这里会把所有节点的相同状态特征函数对应的节点概率相加，特征函数值*概率再加和便是期望。由于特征函数值为1，所以直接加概率值
  }
  for (const_Path_iterator it = lpath.begin(); it != lpath.end(); ++it) { //转移特征的期望
    (*it)->calcExpectation(expected, Z, size);
  }
}
void Path::calcExpectation(double *expected, double Z, size_t size) const {
  const double c = std::exp(lnode->alpha + cost + rnode->beta - Z); //这里计算的是上面提到的边的条件概率P(Yi-1=yi-1,Yi=yi|x)，这里取了exp，跟矩阵形式的计算结果一致
  for (const int *f = fvector; *f != -1; ++f) { 
    expected[*f + lnode->y * size + rnode->y] += c; //这里把所有边上相同的转移特征函数对应的概率相加
  }
}
```

求梯度：

```c++
for (size_t i = 0;   i < x_.size(); ++i) { //遍历每一个位置（词）
    for (const int *f = node_[i][answer_[i]]->fvector; *f != -1; ++f) { //answer_[i]代表当前样本的label，遍历每个词当前样本label的特征，进行减1操作，遍历所有节点减1就相当于公式中fj(y,x)
      --expected[*f + answer_[i]]; //状态特征函数期望减去真实的状态特征函数值
    }
    s += node_[i][answer_[i]]->cost;  // UNIGRAM cost 节点的损失求和
    const std::vector<Path *> &lpath = node_[i][answer_[i]]->lpath;
    for (const_Path_iterator it = lpath.begin(); it != lpath.end(); ++it) {//遍历边，对转移特征做类似计算
      if ((*it)->lnode->y == answer_[(*it)->lnode->x]) {
        for (const int *f = (*it)->fvector; *f != -1; ++f) {
          --expected[*f +(*it)->lnode->y * ysize_ +(*it)->rnode->y]; //转移特征函数期望减去真实转移特征函数值
        }
        s += (*it)->cost;  // BIGRAM COST  边损失求和
        break;
      }
    }
  }
viterbi();  // call for eval() 调用维特比算法做预测，为了计算分类错误的次数，算法详细内容下篇介绍

return Z_ - s ;  //返回似然函数值，看L(w)推导的最后一步，大括号内有两项，其中一项是logZw(x)，我们知道变量Z_是没有取exp的结果，我们要求这一项需要先对Z_取exp，取exp再取log相当于还是Z_，因此 logZw(x) = Z_
                 //再看另一项，是对当前样本得分求和，正好这一项是没有取exp的因此该求和项就等于s， 之前说过CRF++是对似然函数取负号，因此返回Z_ - s
```

 至此，一个句子的似然函数值和梯度就计算完成了。公式的$\sum_1^m$是对所有句子求和，CRF++的求和过程是在run函数调用gradient函数结束后由线程内汇总，然后所有线程结束后再汇总。runCRF函数剩下的代码便是所有线程完成一轮计算后的汇总逻辑，如下：
 
```c++
for (size_t i = 1; i < thread_num; ++i) { //汇总每个线程的数据
      thread[0].obj += thread[i].obj; //似然函数值
      thread[0].err += thread[i].err; 
      thread[0].zeroone += thread[i].zeroone; 
    }

    for (size_t i = 1; i < thread_num; ++i) {
      for (size_t k = 0; k < feature_index->size(); ++k) { 
        thread[0].expected[k] += thread[i].expected[k]; //梯度值求和
      }
    }

    size_t num_nonzero = 0;
    if (orthant) {   // L1 根据L1或L2正则化，更新似然函数值
      for (size_t k = 0; k < feature_index->size(); ++k) {
        thread[0].obj += std::abs(alpha[k] / C);
        if (alpha[k] != 0.0) {
          ++num_nonzero;
        }
      }
    } else { //L2
      num_nonzero = feature_index->size();
      for (size_t k = 0; k < feature_index->size(); ++k) {
        thread[0].obj += (alpha[k] * alpha[k] /(2.0 * C));
        thread[0].expected[k] += alpha[k] / C;
      }
    }

    ...省略代码
    if (lbfgs.optimize(feature_index->size(),
                       &alpha[0],
                       thread[0].obj,
                       &thread[0].expected[0], orthant, C) <= 0) { //传入似然函数值和梯度等参数，调用LBFGS算法
      return false;
    }
```

最终调用LBFGS算法更新w，CRF++中的LBFGS算法最终是调用的Fortran语言编译后的C代码，可读性比较差，本篇文章暂时不深入介绍。至此，一次迭代的计算过程便介绍完毕。